In this project I am using a k-nearest neighbors algorithm to predict a car's market price using its attributes.

# Dataset used in this project
I will be using [UCI's Automobile Data Set](https://archive.ics.uci.edu/ml/datasets/automobile). This data contains car data from the 1985 Ward's Automotive Yearbook and can be downloaded [here](https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data).

##  Variables in the dataset

Variable|Meaning|Type
-|-|-
symboling|Risk symbol (A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe.)|categorical(numeric)
normalized-losses|Normalized losses in use as compared to other cars.|continuous
make|make of car|categorical(string)
fuel-type|diesel or gas|categorical(string)
aspiration|std or turbo|categorical(string)
num-of-doors|four or two|categorical(string)
body-style|hardtop, wagon, sedan, hatchback, convertible|categorical(string)
drive-wheels|4wd, fwd, rwd|categorical(string)
engine-location|front or rear|categorical(string)
wheel-base|horizontal distance between the centers of the front and rear wheels|continuous
length|car length|continuous
width|car width|continuous
height|car height|continuous
curb-weight|total mass of a vehicle with standard equipment and all necessary operating consumables such as motor oil, transmission oil, brake fluid, coolant, air conditioning refrigerant, and sometimes a full tank of fuel, while not loaded with either passengers or cargo.|continuous
engine-type|dohc, dohcv, l, ohc, ohcf, ohcv, rotor.|categorical(string)
num-of-cylinders|eight, five, four, six, three, twelve, two.|categorical(string)
engine-size|size of car engine|continuous
fuel-system|1bbl, 2bbl, 4bbl, idi, mfi, mpfi, spdi, spfi.|categorical(string)
bore|diameter of each cylinder|continuous
stroke|stroke of engine|continuous
compression-ratio|compression ratio of engine|continuous
horsepower|car horsepower|continuous
peak-rpm|peak revolutions per minute|continuous
city-mpg|miles per gallon under city conditions|continuous
highway-mpg|miles per gallon on an open stretch of road|continuous
price|car price|continuous

# Reading in the data

In [1]:
import pandas as pd
import numpy as np

col_names = ["symboling", "normalized-losses", "make", "fuel-type", "aspiration", "num-of-doors", "body-style", "drive-wheels", "engine-location", "wheel-base", "length", "width", "height",
             "curb-weight", "engine-type", "num-of-cylinders", "engine-size", "fuel-system", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg", "price"]
cars = pd.read_csv("imports-85.data", names=col_names)

cars.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450


In [2]:
cars.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

# Cleaning and Preparing Data

* normalized-losses should be int, but is object dtype. Preview already shows likely reason: Missing data encoded as "?".
* bore, stroke, horsepower, peak-rpm and price should be should be int or float, but are object dtype. Possible the same issue as normalized-losses
* num-of-cylinders can be transformed to int

In [3]:
print("Values in bore, alphabetical descending order:")
print(cars["bore"].value_counts().sort_index(ascending=False).head(5))

print("\nValues in stroke, in alphabetical descending order:")
print(cars["stroke"].value_counts().sort_index(ascending=False).head(5))

print("\nValues in horsepower, in alphabetical descending order:")
print(cars["horsepower"].value_counts().sort_index(ascending=False).head(5))

print("\nValues in peak-rpm, in alphabetical descending order:")
print(cars["peak-rpm"].value_counts().sort_index(ascending=False).head(5))

print("\nValues in price, in alphabetical descending order:")
print(cars["price"].value_counts().sort_index(ascending=False).head(5))

Values in bore, alphabetical descending order:
?       4
3.94    2
3.80    2
3.78    8
3.76    1
Name: bore, dtype: int64

Values in stroke, in alphabetical descending order:
?       4
4.17    2
3.90    3
3.86    4
3.64    5
Name: stroke, dtype: int64

Values in horsepower, in alphabetical descending order:
?     2
97    5
95    7
94    2
92    4
Name: horsepower, dtype: int64

Values in peak-rpm, in alphabetical descending order:
?       2
6600    2
6000    9
5900    3
5800    7
Name: peak-rpm, dtype: int64

Values in price, in alphabetical descending order:
?       4
9995    1
9989    1
9988    1
9980    1
Name: price, dtype: int64


As suspected, missing values are coded as "?". Cleaning this up in the next step.

In [4]:
# Record "?" as nan, convert cols to float where appropriate
cols_to_clean = ["normalized-losses", "bore",
                 "stroke", "horsepower", "peak-rpm", "price"]
cars[cols_to_clean] = cars[cols_to_clean].replace("?", np.nan)
cars[cols_to_clean] = cars[cols_to_clean].astype("float")

# Covnert num-of-cylinders to float
cars["num-of-cylinders"].value_counts()

four      159
six        24
five       11
eight       5
two         4
three       1
twelve      1
Name: num-of-cylinders, dtype: int64

In [5]:
string_to_num_dict = {"two": 2, "three": 3, "four": 4,
                      "five": 5, "six": 6, "seven": 7, "eight": 8, "twelve": 12}

cars["num-of-cylinders"].replace(string_to_num_dict, inplace=True)
cars["num-of-cylinders"].value_counts()

4     159
6      24
5      11
8       5
2       4
3       1
12      1
Name: num-of-cylinders, dtype: int64

## Inspecting missing data

In [6]:
print("Missing values for price:")
print(cars["price"].isna().sum())

Missing values for price:
4


Price is the target variable so all rows with missing values for price will be dropped

In [7]:
cars.dropna(subset=["price"], inplace=True)

# Get missing values for all continous variables (except for price, see above)
pot_features = ["normalized-losses", "wheel-base", "length", "width", "height", "curb-weight", "num-of-cylinders",
                "engine-size", "bore", "stroke", "compression-ratio", "horsepower", "peak-rpm", "city-mpg", "highway-mpg"]

print("Total cases:")
print(str(cars.shape[0])+"\n")

print("Missing values:")
cars[pot_features].isna().sum()

Total cases:
201

Missing values:


normalized-losses    37
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
num-of-cylinders      0
engine-size           0
bore                  4
stroke                4
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
dtype: int64

normalized-losses has quite a few missing values (18%) and will therefore be dropped entirely.
For the remaining features, mean imputation will be used.

In [8]:
pot_features.remove("normalized-losses")

cars[pot_features] = cars[pot_features].fillna(cars[pot_features].mean())

## Normalizing Data

In [9]:
# Create new dataframe with only price and normalized continous variables
cars_norm = pd.DataFrame(cars["price"])
cars_norm[pot_features] = (
    cars[pot_features]-cars[pot_features].mean())/(cars[pot_features].std())
cars_norm

,price,wheel-base,length,width,height,curb-weight,num-of-cylinders,engine-size,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg
0,13495.0,-1.680910,-0.438315,-0.851335,-2.029015,-0.014821,-0.342804,0.075201,0.519597,-1.825369,-0.290709,0.203476,-0.245942,-0.650624,-0.540937
1,16500.0,-1.680910,-0.438315,-0.851335,-2.029015,-0.014821,-0.342804,0.075201,0.519597,-1.825369,-0.290709,0.203476,-0.245942,-0.650624,-0.540937
2,16500.0,-0.708334,-0.243544,-0.185134,-0.558319,0.516789,1.544965,0.604724,-2.427374,0.674254,-0.290709,1.354267,-0.245942,-0.961995,-0.687669
3,13950.0,0.165335,0.194690,0.147966,0.217881,-0.422710,-0.342804,-0.430252,-0.524899,0.452769,-0.041019,-0.037387,0.799836,-0.183569,-0.100741
4,17450.0,0.099398,0.194690,0.243137,0.217881,0.518722,0.601081,0.219617,-0.524899,0.452769,-0.540399,0.310527,0.799836,-1.117680,-1.274597
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,16845.0,1.698379,1.184775,1.432780,0.708112,0.766162,-0.342804,0.339963,1.676004,-0.338251,-0.165864,0.283764,0.590680,-0.339254,-0.394205
201,19045.0,1.698379,1.184775,1.385194,0.708112,0.953676,-0.342804,0.339963,1.676004,-0.338251,-0.365616,1.514843,0.381525,-0.961995,-0.834401
202,21485.0,1.698379,1.184775,1.432780,0.708112,0.882150,1.544965,1.110178,0.929935,-1.224194,-0.340647,0.819016,0.799836,-1.117680,-1.127865
203,22470.0,1.698379,1.184775,1.432780,0.708112,1.278441,1.544965,0.436240,-1.196361,0.452769,3.204952,0.069663,-0.664253,0.127801,-0.540937


# First Testing: Univariate KNN models

In [10]:
def knn_train_test(train_col, target_col, df):
    return "coming soon..."